In [1]:
import pandas as pd
import pickle 
from scipy.stats import poisson


In [2]:
dict_table =pickle.load(open('dict_table', 'rb'))
df_data_historica=pd.read_csv('clean_fifa_worldcup_matches.csv')
df_fixture=pd.read_csv('clean_fifa_worldcup_fixture.csv')

In [3]:
df_data_historica.head()

,HomeTeams,AwayTeams,Year,HomeGoals,AwayGoals
0,Yugoslavia,Brazil,1930,2,1
1,Uruguay,Yugoslavia,1930,6,1
2,Argentina,United States,1930,6,1
3,Paraguay,Belgium,1930,1,0
4,United States,Paraguay,1930,3,0


In [4]:
df_home=df_data_historica[['HomeTeams','HomeGoals','AwayGoals']]
df_away=df_data_historica[['AwayTeams','HomeGoals','AwayGoals']]

In [5]:
df_home=df_home.rename(columns={'HomeTeams':'Team','HomeGoals':'GoalsScored','AwayGoals':'GoalsConsede'})
df_away=df_away.rename(columns={'AwayTeams':'Team','HomeGoals':'GoalsConsede','AwayGoals':'GoalsScored'})
df_away

,Team,GoalsConsede,GoalsScored
0,Brazil,2,1
1,Yugoslavia,6,1
2,United States,6,1
3,Belgium,1,0
4,Paraguay,3,0
...,...,...,...
895,Costa Rica,2,0
896,Switzerland,1,2
897,Brazil,0,2
898,Mexico,0,1


In [6]:
df_poderio=pd.concat([df_home,df_away],ignore_index=True).groupby('Team').mean()

# funcion de prediction

In [7]:
def predict_poisson(home,away):
    if home in df_poderio.index and away in df_poderio.index:
        lamb_home=df_poderio.at[home,'GoalsScored']* df_poderio.at[away,'GoalsConsede']
        lamb_away=df_poderio.at[away,'GoalsScored']* df_poderio.at[home,'GoalsConsede']
        prob_home,prob_away,prob_draw=0,0,0
        for x in range(0,11):
            for y in range(0,11):
                p=poisson.pmf(x,lamb_home)*poisson.pmf(y,lamb_away)
                if x==y:
                    prob_draw+=p
                elif x>y:
                    prob_home+=p
                else:
                    prob_away+=p
        
        points_home=3 * prob_home + prob_draw
        points_away=3 * prob_away + prob_draw
        return (points_home,points_away)
    else:
        return (0,0)


In [8]:
predict_poisson('United States','Mexico')

(1.3659104461490816, 1.4243775371296254)

In [9]:
df_fixture_grup_48=df_fixture[:48].copy()
df_fixture_octavos=df_fixture[48:56].copy()
df_fixture_cuarter=df_fixture[56:60].copy()
df_fixture_semi=df_fixture[60:62].copy()
df_fixture_final=df_fixture[62:].copy()

In [10]:
for group in dict_table:
    teams_in_group = dict_table[group]['Team'].values
    df_fixture_group_6 = df_fixture_grup_48[df_fixture_grup_48['home'].isin(teams_in_group)]
    for index, row in df_fixture_group_6.iterrows():
        home, away = row['home'], row['away']
        points_home, points_away = predict_poisson(home, away)
        dict_table [group].loc[dict_table[group]['Team']== home, 'Pts'] += points_home
        dict_table [group].loc[dict_table[group]['Team'] == away, 'Pts'] += points_away

        
    dict_table[group] = dict_table[group].sort_values('Pts', ascending=False).reset_index()
    dict_table[group] = dict_table[group][['Team', 'Pts']]
    dict_table[group] = dict_table[group].round(0)


In [11]:
df_fixture_octavos

,home,score,away,year
48,Winners Group A,Match 49,Runners-up Group B,2022
49,Winners Group C,Match 50,Runners-up Group D,2022
50,Winners Group D,Match 52,Runners-up Group C,2022
51,Winners Group B,Match 51,Runners-up Group A,2022
52,Winners Group E,Match 53,Runners-up Group F,2022
53,Winners Group G,Match 54,Runners-up Group H,2022
54,Winners Group F,Match 55,Runners-up Group E,2022
55,Winners Group H,Match 56,Runners-up Group G,2022


In [12]:
for group in dict_table:
    group_winner=dict_table[group].loc[0,'Team']
    runners=dict_table[group].loc[1,'Team']
    df_fixture_octavos.replace({f'Winners {group}':group_winner,
                                f'Runners-up {group}':runners},inplace=True)



In [13]:
df_fixture_octavos['Winner']='?'


In [14]:
def get_winner(df_fixture_updated):
    for index ,row in df_fixture_updated.iterrows():
        home,awya= row['home'],row['away']
        points_home,points_away= predict_poisson(home,away)
        if points_home > points_away:
            winner=home
        else:
            winner=away
        df_fixture_updated.loc[index, 'Winner']=winner
    return df_fixture_updated


In [15]:
get_winner(df_fixture_octavos)

,home,score,away,year,Winner
48,Netherlands,Match 49,Wales,2022,Netherlands
49,Argentina,Match 50,Denmark,2022,Argentina
50,France,Match 52,Poland,2022,France
51,England,Match 51,Senegal,2022,England
52,Germany,Match 53,Belgium,2022,Germany
53,Brazil,Match 54,Uruguay,2022,Brazil
54,Croatia,Match 55,Spain,2022,Portugal
55,Portugal,Match 56,Switzerland,2022,Portugal


In [16]:
def update_table(df_fixture_round_1,df_fixture_round_2):
    for index, row in df_fixture_round_1.iterrows():
        winner=df_fixture_round_1.loc[index,'Winner']
        math=df_fixture_round_1.loc[index,'score']
        df_fixture_round_2.replace({f'Winners {math}': winner},inplace=True)
    df_fixture_round_2['Winner']='?'
    return df_fixture_round_2

In [17]:
update_table(df_fixture_octavos,df_fixture_cuarter)

,home,score,away,year,Winner
56,Germany,Match 58,Brazil,2022,?
57,Netherlands,Match 57,Argentina,2022,?
58,Portugal,Match 60,Portugal,2022,?
59,England,Match 59,France,2022,?


In [18]:
get_winner(df_fixture_cuarter)

,home,score,away,year,Winner
56,Germany,Match 58,Brazil,2022,Germany
57,Netherlands,Match 57,Argentina,2022,Netherlands
58,Portugal,Match 60,Portugal,2022,Portugal
59,England,Match 59,France,2022,England


In [19]:
update_table(df_fixture_cuarter,df_fixture_semi)

,home,score,away,year,Winner
60,Netherlands,Match 61,Germany,2022,?
61,England,Match 62,Portugal,2022,?


In [20]:
get_winner(df_fixture_semi)

,home,score,away,year,Winner
60,Netherlands,Match 61,Germany,2022,Netherlands
61,England,Match 62,Portugal,2022,England


In [21]:
update_table(df_fixture_semi,df_fixture_final)

,home,score,away,year,Winner
62,Losers Match 61,Match 63,Losers Match 62,2022,?
63,Netherlands,Match 64,England,2022,?


In [22]:
get_winner(df_fixture_final)

,home,score,away,year,Winner
62,Losers Match 61,Match 63,Losers Match 62,2022,Portugal
63,Netherlands,Match 64,England,2022,Netherlands
